# Simulated Data
Environment check: numpy, pandas, xlsx support.

In [1]:
import numpy as np
import pandas as pd

print(f"numpy  {np.__version__}")
print(f"pandas {pd.__version__}")

numpy  2.4.2
pandas 3.0.1


In [2]:
# Verify xlsx read/write via openpyxl
df = pd.DataFrame({
    "x": np.arange(5),
    "y": np.random.randn(5)
})

df.to_excel("test_output.xlsx", index=False)
df_back = pd.read_excel("test_output.xlsx")

print("xlsx round-trip OK")
df_back

xlsx round-trip OK


,x,y
0,0,0.049950
1,1,-0.442808
2,2,-1.173784
3,3,-0.760213
4,4,0.636042
